# Explore FVCOM GOM3 Hindcast on AWS Public Data

In [1]:
import xarray as xr
import zarr
import intake

In [2]:
intake_catalog_url = 'https://umassd-fvcom.s3.amazonaws.com/gom3/hindcast/umassd-fvcom-gom3.yml'
cat = intake.open_catalog(intake_catalog_url)

In [3]:
list(cat)

['UMASSD-FVCOM-GOM3-zarr2', 'UMASSD-FVCOM-GOM3-zarr3']

In [4]:
if zarr.__version__[0]=='2':
    dataset = 'UMASSD-FVCOM-GOM3-zarr2' 
elif zarr.__version__[0]=='3':
    dataset = 'UMASSD-FVCOM-GOM3-zarr3'

In [5]:
%%time
ds = cat[dataset].to_dask()

CPU times: user 2.38 s, sys: 1.1 s, total: 3.47 s
Wall time: 4.69 s


In [6]:
ds

<xarray.Dataset> Size: 45TB
Dimensions:             (time: 341881, node: 48451, four: 4, nele: 90415,
                         three: 3, siglev: 46, maxnode: 11, maxelem: 9,
                         siglay: 45)
Coordinates:
    siglay              (siglay, node) float32 9MB dask.array<chunksize=(45, 48451), meta=np.ndarray>
    siglev              (siglev, node) float32 9MB dask.array<chunksize=(46, 48451), meta=np.ndarray>
  * time                (time) datetime64[ns] 3MB 1978-01-01 ... 2017-01-01
    x                   (node) float32 194kB dask.array<chunksize=(48451,), meta=np.ndarray>
    y                   (node) float32 194kB dask.array<chunksize=(48451,), meta=np.ndarray>
Dimensions without coordinates: node, four, nele, three, maxnode, maxelem
Data variables: (12/49)
    ML_Depth            (time, node) float32 66GB dask.array<chunksize=(1, 48451), meta=np.ndarray>
    ML_Nsiglay          (time, node) int32 66GB dask.array<chunksize=(1, 48451), meta=np.ndarray>
    a1u                 (four, nele) float32 1MB dask.array<chunksize=(4, 90415), meta=np.ndarray>
    a2u                 (four, nele) float32 1MB dask.array<chunksize=(4, 90415), meta=np.ndarray>
    art1                (node) float32 194kB dask.array<chunksize=(48451,), meta=np.ndarray>
    art2                (node) float32 194kB dask.array<chunksize=(48451,), meta=np.ndarray>
    ...                  ...
    wet_nodes           (time, node) int32 66GB dask.array<chunksize=(1, 48451), meta=np.ndarray>
    wet_nodes_prev_int  (time, node) int32 66GB dask.array<chunksize=(1, 48451), meta=np.ndarray>
    ww                  (time, siglay, nele) float32 6TB dask.array<chunksize=(1, 45, 90415), meta=np.ndarray>
    xc                  (nele) float32 362kB dask.array<chunksize=(90415,), meta=np.ndarray>
    yc                  (nele) float32 362kB dask.array<chunksize=(90415,), meta=np.ndarray>
    zeta                (time, node) float32 66GB dask.array<chunksize=(1, 48451), meta=np.ndarray>
Attributes: (12/14)
    title:                       GOM3 Nesting  by Yf.Sun@umassd.edu
    institution:                 School for Marine Science and Technology
    source:                      FVCOM_3.0
    history:                     model started at: 22/12/2011   16:41
    references:                  http://fvcom.smast.umassd.edu, http://codfis...
    Conventions:                 CF-1.0
    ...                          ...
    Tidal_Forcing:               TIDAL ELEVATION FORCING IS OFF!
    River_Forcing:               THERE ARE 50 RIVERS IN THIS MODEL.\nRIVER IN...
    GroundWater_Forcing:         GROUND WATER FORCING IS OFF!
    Surface_Heat_Forcing:        FVCOM variable surface heat forcing file:\nF...
    Surface_Wind_Forcing:        FVCOM variable surface Wind forcing:\nFILE N...
    Surface_PrecipEvap_Forcing:  FVCOM periodic surface precip forcing:\nFILE...

## Visualize data on the triangular mesh

In [7]:
import numpy as np
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
import hvplot.xarray
import holoviews.operation.datashader as dshade
import pandas as pd

dshade.datashade.precompute = True
hv.extension('bokeh')

In [8]:
var_name='temp'  # variable
level = 0  # vertical level (0 is surface, -1 is bottom)
time = '2015-11-01 04:00'  # time in UTC

In [9]:
%%time
values = ds[var_name].isel(siglay=0).sel(time=time, method='nearest').load()

CPU times: user 2.8 s, sys: 748 ms, total: 3.55 s
Wall time: 5.11 s


In [10]:
v = np.vstack((ds['lon'], ds['lat'], values)).T
verts = pd.DataFrame(v, columns=['lon','lat','var'])

In [11]:
points = gv.operation.project_points(gv.Points(verts, vdims=['var']))

In [12]:
tris = pd.DataFrame(ds['nv'].T.values.astype('int')-1, columns=['v0','v1','v2'])

In [13]:
tiles = gv.tile_sources.OSM

In [14]:
title = f'{var_name}@level {level}   time:{time}'

In [15]:
trimesh = gv.TriMesh((tris, points), label=title)
mesh = dshade.rasterize(trimesh).opts(
              cmap='rainbow', colorbar=True, width=600, height=400)

In [16]:
tiles * mesh

:DynamicMap   []
   :Overlay
      .WMTS.I                                                                                       :WMTS   [Longitude,Latitude]
      .Image.Temp_commercial_at_level_0_time_colon_2015_hyphen_minus_11_hyphen_minus_01_04_colon_00 :Image   [lon,lat]   (lon_lat var)